Классы для работы с seg- и аудиофайлами

In [ ]:
import wave
import struct

def detect_encoding(file_path):
    encoding = "utf-8"
    try:
        l = open(file_path, 'r', encoding="utf-8").read()
        if l.startswith("\ufeff"):
            encoding = "utf-8-sig"
    except UnicodeDecodeError:
        try:
            open(file_path, 'r', encoding="utf-16").read()
            encoding = "utf-16"
        except UnicodeError:
            encoding = "cp1251"
    return encoding

from itertools import product
letters = "GBRY"
nums = "1234"
levels = [ch + num for num, ch in product(nums, letters)]
level_codes = [2 ** i for i in range(len(levels))]

level2code = {i: j for i, j in zip(levels, level_codes)}
code2level = {j: i for i, j in zip(levels, level_codes)}

sampwidth_to_char = {1: "c", 2: "h", 4: "i"}

In [ ]:
class Params:
  def __init__(self, s_rate, s_width, n_channels) -> None:
      self.samplerate = s_rate
      self.sampwidth = s_width
      self.numchannels = n_channels

In [ ]:
class Label:
  def __init__(self, position, level, text) -> None:
    self.position = position
    self.level = level
    self.text = text

In [ ]:
class Seg:
  def __init__(self, filename: str = None, labels: list = [], params: Params = Params(22050, 2, 1)):
    self.filename = filename
    self.labels = labels
    self.params = params

  def read_seg_file(self):
    try:
      with open(self.filename, "r", encoding=detect_encoding(self.filename)) as f:
        lines = [line.strip() for line in f.readlines()]
    except FileNotFoundError:
      print(self.filename, " не найден")

    self.init_params()

    try:
      index_labels = lines.index('[LABELS]')
    except ValueError:
      print("Seg-файл не содержит секции LABELS")

    labels_ = lines[index_labels + 1 :]
    labels_arr = [Label(
        int(line.split(",")[0]) // self.params.sampwidth // self.params.numchannels,
        code2level[int(line.split(",")[1])],
        line.split(",")[2]
    ) for line in labels_ if line.count(",") >= 2]

    self.labels = labels_arr

  def init_params(self):
    try:
      with open(self.filename, "r", encoding=detect_encoding(self.filename)) as f:
        lines = [line.strip() for line in f.readlines()]
    except FileNotFoundError:
      print(self.filename, " не найден")

    try:
      index_params = lines.index('[PARAMETERS]')
    except ValueError:
      print("Seg-файл не содержит секции PARAMETERS")

    try:
      index_labels = lines.index('[LABELS]')
    except ValueError:
      print("Seg-файл не содержит секции LABELS")

    parameters = lines[index_params + 1 : index_labels]

    param_dict = {str(line.split("=")[0]): int(line.split("=")[1]) for line in parameters}

    self.params = Params(param_dict["SAMPLING_FREQ"], param_dict["BYTE_PER_SAMPLE"], param_dict["N_CHANNEL"])

In [ ]:
class Signal:
  def __init__(self, filename: str, signal: list = [], params: Params = None, seg: Seg = None):
    self.signal: list = signal
    self.filename: str = filename
    self.seg: Seg = seg
    self.params: Params = params

  def init_params(self):
    if self.params is not None:
      return
    if self.seg is not None:
      self.seg.init_params()
      self.params = self.seg.params
    else:
      default_params = Params(22050, 2, 1)
      self.params = default_params

  def read_sbl(self):
    with open(self.filename, "rb") as f:
      raw_signal = f.read()
    num_samples = len(raw_signal) // self.params.sampwidth
    fmt = str(num_samples) + sampwidth_to_char[self.params.sampwidth]
    signal = struct.unpack(fmt, raw_signal)
    self.signal = signal

Для выполнения заданий вам могут понадобиться следующие пакеты библиотеки scipy:

In [ ]:
from scipy import signal # siganl.get_window
from scipy.fft import fft, fftfreq